# Heterodyne Source

This instrument is meant to represent a two tone heterodyne source comprised either of two microwave sources or one with dual output (these should be input via the microwave source interface driver which means they can of course be simulated). It can also be simulated with no underlying microwave sources. Since I trust that the microwave source interface works I use only simulated microwave sources and a simulated heterodyne source.<br><br>
Note that for a single source heterodyne source the microwave source must have both the dual utput option and the IQ option.

In [1]:
from qcodes import Instrument
from qdev_wrappers.customised_instruments.interfaces.microwave_source_interface import SimulatedMicrowaveSourceInterface
from qdev_wrappers.customised_instruments.composite_instruments.heterodyne_source.heterodyne_source import SimulatedHeterodyneSource, OneSourceHeterodyneSource, TwoSourceHeterodyneSource

/Users/natalie/Documents/PhD/Qdev/QdevWrappers/qdev_wrappers/logger.py:16: UserWarning: The logger.py of qdev-wrappers is deprecated and will be removed soon. Please use the logger of QCoDeS instead.
  warnings.warn('The logger.py of qdev-wrappers is deprecated and will be '


In [2]:
def print_params(*instruments):
    for instr in instruments:
        if instr is not None:
            print(instr.name.upper())
            for k, v in instr.parameters.items():
                if k != 'IDN':
                    print(k, ': ', v())
            print('\n')

In [3]:
# choose from 0 (simulates heterodyne_source), 1 (uses one simulated microwave source),
#             2 (uses two simulated microwave sources)
num_sources = 2

In [5]:
try:
    Instrument.remove_instance(heterodyne_source)
    Instrument.remove_instance(carrier_source_if)
    Instrument.remove_instance(localos_source_if)
except (NameError, TypeError):
    pass
carrier_source_if = None
localos_source_if = None
if num_sources == 0:
    heterodyne_source = SimulatedHeterodyneSource('heterodyne_source')
elif num_sources == 1:
    carrier_source_if = SimulatedMicrowaveSourceInterface('microwave_source', IQ=True, dual_output=True)
    heterodyne_source = OneSourceHeterodyneSource('heterodyne_source', carrier_source_if, localos_power=10)
elif num_sources == 2:
    carrier_source_if = SimulatedMicrowaveSourceInterface('carrier_microwave_source', IQ=False)
    localos_source_if = SimulatedMicrowaveSourceInterface('localos_microwave_source', IQ=False)
    heterodyne_source = TwoSourceHeterodyneSource('heterodyne_source', carrier_source_if, localos_source_if)

print_params(heterodyne_source, carrier_source_if, localos_source_if)

HETERODYNE_SOURCE
frequency :  7000000000.0
power :  -10
demodulation_frequency :  0.0
localos_power :  -10
status :  0
mode :  basic


CARRIER_MICROWAVE_SOURCE
frequency :  7000000000.0
power :  -10
status :  0
pulsemod_state :  0
IQ_state :  0
dual_output_state :  0


LOCALOS_MICROWAVE_SOURCE
frequency :  7000000000.0
power :  -10
status :  0
pulsemod_state :  0
IQ_state :  0
dual_output_state :  0




Have a play with changing the parameters. Especially note the modes available (which depend on what you are using to make up your heterodyne source):
- basic: IQ state off, pulsemod state off, second output (localos or dual output state) off
- sidebanded: IQ state on, pulsemod state off, second output matching main status
- modulated: IQ state off, pulsemod state on, second output matching main status
- sidebanded_modulated: IQ state on, pulsemod state on, , second output matching main status
Also note that demodulation_frequency is not available for a single source heterodyne source and that the localos_power for this implementation is fixed by the dual output power and cannot be set

In [6]:
try:
    heterodyne_source.frequency(6e9)
    heterodyne_source.power(-20)
    heterodyne_source.demodulation_frequency(15e6)
    heterodyne_source.localos_power(15)
    heterodyne_source.status(1)
    heterodyne_source.mode('sidebanded_modulated')
except Exception as e:
    print('ERROR: {} \n'.format(e))

print_params(heterodyne_source, carrier_source_if, localos_source_if)

ERROR: ("'sidebanded_modulated' is not in {'modulated', 'basic'}; Parameter: heterodyne_source.mode", 'setting heterodyne_source_mode to sidebanded_modulated') 

HETERODYNE_SOURCE
frequency :  6000000000.0
power :  -20
demodulation_frequency :  15000000.0
localos_power :  15
status :  1
mode :  basic


CARRIER_MICROWAVE_SOURCE
frequency :  6000000000.0
power :  -20
status :  1
pulsemod_state :  0
IQ_state :  0
dual_output_state :  0


LOCALOS_MICROWAVE_SOURCE
frequency :  6015000000.0
power :  15
status :  0
pulsemod_state :  0
IQ_state :  0
dual_output_state :  0


